# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import gmaps
from pprint import pprint
import json

# Import API key
from api_keys import geoapify_key
gmaps.configure(api_key = geoapify_key)

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")




In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_df = pd.read_csv("output_data/cities.csv")


# Display sample data
city_df.head()

In [ ]:

city_df["Humidity"] = city_df["Humidity"].astype(int)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
!jupyter nbextension enable --py gmaps



In [ ]:
import folium

# Define the map center and zoom level
center = (20.0, 0.0)
zoom_level = 2

# Create the map
m = folium.Map(location=center, zoom_start=zoom_level)

# Add markers to the map
for _, row in city_df.iterrows():
    popup = f"City: {row['City']}<br>Country: {row['Country']}<br>Humidity: {row['Humidity']}"
    folium.Marker([row['Lat'], row['Lng']], popup=popup, icon=folium.Icon(color='blue')).add_to(m)

# Save the map as an HTML file
m.save('my_map.html')


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
good_weather_df = city_df.dropna()

# Display sample data
good_weather_df


In [ ]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

good_weather_df = pd.DataFrame(good_weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (ideal_weather_df["Max Temp"] <= 80) & (ideal_weather_df["Max Temp"] > 70)
wind_speed = ideal_weather_df["Wind Speed"] < 10
cloudiness = ideal_weather_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
good_weather_df[max_temp & wind_speed & cloudiness]


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_df [['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df["Hotel Name"] = ""

hotel_df


In [ ]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
#hotel_df['Hotel Name'] = pd.Series(dtype=str)

# display the updated DataFrame to verify the new column was added
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Define the API endpoint and parameters
url = "https://api.geoapify.com/v2/places"
params = {
    "categories": "accommodation.hotel",
    "limit": 10,
    "apiKey": geoapify_key
}

for i in range(len(hotel_df)):
    Lat = hotel_df.loc[i, "Lat"]
    Lon = hotel_df.loc[i, "Lng"]
    params["filter"]=f"circle:{Lon},{Lat},10000"
    params["bias"] = f"proximity:{Lon},{Lat}"
    response = requests.get(url, params=params).json()
    try:
        hotel_name = response["features"][0]["properties"]["name"]
        hotel_df.loc[i, "Hotel Name"] = hotel_name
    except:
        hotel_df.loc[i, "Hotel Name"] = "No Hotel Found"
    print(f"{hotel_df.loc[i,'Hotel Name']}")


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
import plotly.graph_objs as go

# Initialize the figure with the mapbox style and center it on the first city in the city_df DataFrame
fig = go.Figure(go.Scattermapbox(
        lat=city_df["Lat"],
        lon=city_df["Lng"],
        mode='markers',
        marker=dict(size=10, color='blue'),
        text=city_data_df["City"],
        hovertemplate="<b>%{text}</b><br><br>" +
                      "Lat: %{lat:.2f}<br>" +
                      "Lng: %{lng:.2f}<br>" +
                      "<extra></extra>"
))

# Create a list to hold the hover text
hover_text = []

# Iterate through the hotel_df DataFrame and add the hotel name and country to the hover text
for index, row in hotel_df.iterrows():
    hover_text.append(f"{row['Hotel Name']}, {row['Country']}")

# Create a Scattermapbox trace for the hotels
hotel_trace = go.Scattermapbox(
    lat=hotel_df["Lat"],
    lon=hotel_df["Lng"],
    mode="markers",
    marker=dict(
        size=10,
        color="blue",
    ),
    text=hover_text, # Use the hover_text list for the hover text
)

# Add the trace to the figure and show the figure
fig.add_trace(hotel_trace)
fig.show()


In [ ]:
import plotly.graph_objs as go

# Create a list to hold the hover text
hover_text = []

# Iterate through the hotel_df DataFrame and add the hotel name and country to the hover text
for index, row in hotel_df.iterrows():
    hover_text.append(f"{row['Hotel Name']}, {row['Country']}")

# Create a Scattermapbox trace for the hotels
hotel_trace = go.Scattermapbox(
    lat=hotel_df["Lat"],
    lon=hotel_df["Lng"],
    marker=dict(
        size=10,
        color="red",
    ),
    text=hover_text, # Use the hover_text list for the hover text
)

# Add the trace to the figure and show the figure
fig.add_trace(hotel_trace)
fig.show()
